# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Dataset, Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import joblib
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
ws = Workspace.from_config()
experiment_name = 'exp2'
experiment=Experiment(ws, experiment_name)
data = Dataset.get_by_name(ws, name="housedata")

In [3]:
try:
    cluster = ComputeTarget(workspace=ws, name="compute11")
    print("Cluster exists")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2', max_nodes=4)
    cluster = ComputeTarget.create(ws, "compute11", config)

cluster.wait_for_completion()

Cluster exists


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--lrate": choice(0.001, 0.01, 0.1, 1, 10, 100, 200), "--niter": choice(200, 500, 1000)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=cluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name="MSE", primary_metric_goal=PrimaryMetricGoal.MINIMIZE, max_total_runs=10, max_concurrent_runs=4)

In [20]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best = hyperdrive_run.get_best_run_by_primary_metric()
best_metric = best.get_metrics()
param = best.get_details()['runDefinition']
#print("Best metric: {}".format(best_metric['Accuracy']))
#print("Best paramters: {}".format(param))

In [ ]:
best.register_model("best_model", "outputs/project_model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service